#Subprocess moudle tutorial

Reference from [**PyMOTW**](http://pymotw.com/2/subprocess/index.html).

The **`subprocess`** module provides a consistent interface to creating and working with additional processes. It offers a higher-level interface thatn some of the other available modules, and is intended to replace functions such as **`os.system()`**, **`os.spawn*()`**, **`os.popen*()`**, and **`commands.*()`**. To make it easier to compare **`subprocess`** with those other modules, many of the example here re-create the ones used for **`os`** and popen.

The **`subprocess`** module defines one class, **`Popen`** and a few wrapper functions that use that class. The constructor for **`Popen`** takes arguments to set up the new process so the parent can communicate with it via pipes. It provides all of the functionality of the other modules and functions it replaces, and more. The API is consistence for all uses, and many of the extra steps of overhead needed (such as closing file descriptors and ensuring the pipes are closed) are "built in" instead of being handled by the application code separately.

#Running External Command

To run an external command without interacting with it, such as one would do with [*os.system()*](http://pymotw.com/2/os/index.html#os-system), use the **`call()`** function.

In [ ]:
import subprocess

# Simple command
subprocess.call(['ls', '-l'], shell=False)

**Note:** The IPython notebook can only capture the output in the current process.

Setting the *shell* argument to a true value causes **`subproces`** to spawn an intermediate shell process, and tell it to run the command. The default is to run the command directly. Please see the [*thread*](http://stackoverflow.com/questions/11566967/python-raise-child-exception-oserror-errno-2-no-such-file-or-directory) on stackoverflow.

In [ ]:
import subprocess

# Command with shell expansion
subprocess.call('echo $HOME', shell=True)

#Error Handling

The return value from **`call()`** is the exit code of the program. The caller is responsible for interpreting it to detect errors. The **`check_call()`** function works like **`call()`** except that the exit code is checked, and if it indicates an error happened then a **`CalledProcessError`** exception is raised.

In [ ]:
import subprocess

subprocess.check_call(['false'])

##Capturing Output
The standard input and output channels for the process started by call() are bound to the parent's input and output. That means the calling program cannot capture the output of the command. Use **`check_ouput()`** to capture the output for later processing.

In [ ]:
import subprocess

output = subprocess.check_output(['ls', '-l'])
print 'Have %d bytes in output' % len(output)
print output

In [ ]:
import subprocess
output = subprocess.check_output(
    'echo to stdout; echo to stderr 1>&2; exit 1',
    shell = True,
    )
print 'Have %d bytes in output' % len(output)
print output

**Note:** When the subprocess exit without 0, a exception will be thrown.

#Working with Pipes Directly

By passing different arguments for *`stdin`*, *`stdout`*, and *`stderr`* it is possible to mimic the variations of **`os.popen()`**.

##popen

To run a process and read all of its ouput, set the *`stdout`* value to **`PIPE`** and call **`communicate()`**.

In [ ]:
import subprocess

print '\nread:'
proc = subprocess.Popen(['echo', '"to stdout"'],
                       stdout = subprocess.PIPE,
                       )
# communicate return (stdout, stderr)
stdout_value = proc.communicate()[0]
print '\tstdout:', repr(stdout_value)

To send data to the standard input channel of the process one time, pass the data to **`communicate()`**. This is similar to using **`popen()`** with mode '**`w`**'.

In [ ]:
import subprocess

print '\nwrite:'
proc = subprocess.Popen(['cat','-'],
                        stdin=subprocess.PIPE,
                       )
proc.communicate('\tstdin: Hello, Sean!\n')
print "You won't see any thing because of IPython notebook(see above note.)"

##popen2

To set up the **`Popen`** instance for reading and writing, use a combination of the previous techniques.

In [ ]:
import subprocess

print '\npopen2'

proc = subprocess.Popen(['cat','-'],
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                       )
stdout_value = proc.communicate('through stdin to stdout')[0]
print '\tpass through:', repr(stdout_value)

##popen3

It is also possible watch both of the streams of stdout and stderr.


In [ ]:
import subprocess

print '\npopen3'

proc = subprocess.Popen('cat -; echo "to stderr" 1>&2',  # 1>&2 redirect stdout to stderr
                        shell=True,                      # Important!
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                       )
stdout_value, stderr_value = proc.communicate('through stdin to stdout')
print '\tpass through:', repr(stdout_value)
print '\tstderr      :', repr(stderr_value)

##popen4

To direct the error output from the process to its standard ouput channel, use **`STDOUT`** for *`stderr`* instead of **`PIPE`**.

In [ ]:
import subprocess

print '\npopen4:'
proc = subprocess.Popen('cat -; echo "to stderr" 1>&2',
                        shell=True,
                        stdin=subprocess.PIPE,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.STDOUT,
                       )
stdout_value, stderr_value = proc.communicate('through stdin to stdout\n')
print '\tcombined output:', repr(stdout_value)
print '\tstderr value   :', repr(stderr_value)

#Connecting Segments of a Pipe

Mutiple commands can be connected into a *pipeline*, similar to the way the Unix shell works, by creating separate **`Popen`** instances and chaining their inputs and outputs together. The **`stdout`** attribute of one **`Popen`** instance is used as the *stdin* argument for the next in the pipeline, instead of the constant **PIPE**. The output is read from the **`stdout`** handle for the final command in the pipeline.

In [ ]:
import subprocess
cat = subprocess.Popen(['cat', 'test.txt'],
                       stdout=subprocess.PIPE,
                       )

grep1 = subprocess.Popen(['grep', 'Sean'],
                        stdin=cat.stdout,
                        stdout=subprocess.PIPE,
                       )
grep2 = subprocess.Popen(['grep', 'Lan'],
                         stdin=grep1.stdout,
                         stdout=subprocess.PIPE,
                         )
grep1_stdout = grep1.stdout
grep2_stdout = grep2.stdout

# print 'grep1 stdout:'
# for line in grep1_stdout:
#    print '\t', line.strip()

print 'grep2 stdout:'
for line in grep2_stdout:
    print '\t', line.strip()
                         

#Interacting with Another Command

All of the above examples assume a limited amount of interaction. The **`communicate()`** method reads all of the output and waits for child process to exit before returning. It is also possible to write to and read from the individual pipe handles used by the **`Popen`** instance. A simple echo program that reads from standard input and writes to standard output illustrates this:

In [ ]:
# %load repeater.py
#!/usr/bin/python

import sys
sys.stderr.write('repeater.py: starting\n')
sys.stderr.flush()

while True:
    next_line = sys.stdin.readline()
    if not next_line:
        break
    sys.stdout.write(next_line)
    sys.stdout.flush()

sys.stderr.write('repeater.py: exiting\n')
sys.stderr.flush()


The script, `repeater.py`, writes to stderr when it starts and stops. That information can be used to show the lifetime of the child process.  
**Note:** Please run this example in the terminal

The next interaction example uses the stdin and stdout file handles owned by the **`Popen`** instance in different ways. In the first example, a sequence of 10 numbers are written to stdin of the process, and after each write the next line of output is read back. In the second example, the same 10 numbers are written but the output is read all at once using **`communicate()`**.

In [ ]:
%load subprocess_repeater.py

#Signaling Between Processes

Since each Popen instance provides a *pid* attribute with the process id of the child process, it is possible to do some signaling between processes. For example, using this script for the child process to be executed by the parent process

In [ ]:
%load signal_child.py

combined with this parent process

In [ ]:
%load subprocess_signal.py

The output in shell terminal is as following:

In [ ]:
from IPython.display import Image
Image(filename='images/signal.jpg')

##Process Groups / Sessions

Because of the way the process tree works under UNIX, if the process created by **`Popen`** spawns sub-processes, those children will not receive any signals sent to the parent. That means, for example, it will be difficult to cause them to terminate by sending **`SIGINT`** or **`SIGTERM`**.

In [ ]:
%load subprocess_signal_parent_shell.py

The pid used to send the signal does not match the pid of the child of the shell script waiting for the signal because in this example, there are three separate process interacting:
1. `subprocess_signal_parent_shell.py`
2. The Unix shell process running the script created by the main python program.
3. `signal_child.py`

In [ ]:
from IPython.display import Image
Image(filename='images/signal_parent_shell.jpg')

The solution to this problem is to use a *process group* to associate the children so they can be signaled together. The process group is created with **`os.setsid()`**, settin gthe "session id" to the process id of the current process. All child processes inherit the session id, and since it should only be set in the shell created by **`Popen`** and its descentants, **`os.setsid()`** should not be called in the parent process. Instead, the function is passed to **`Popen`** as the *`preexec_fn`* argument so it is run after the **`fork()`** inside the new process, before it uses **`exec()`** to run the shell.

In [ ]:
%load subprocess_signal_parent_group_shell.py

The sequence of events is:
   1. The parent program instantiates **`Popen`**.
   2. The **`Popen`** instance forks a new process.
   3. The new process runs **`os.setsid()`**.
   4. The new process runs **`exec()`** to start the shell.
   5. The shell runs the shell script.
   6. The shell script forks again and that process execs Python.
   7. Python runs `signal_child.py`.
   8. The parent program signals the process group using the pid of the shell.
   9. The shell and Python process receive the signal. The shell ignores it. Python invokes the signal handler.

To singal the entire process group, use **`os.killpg()`** with the pid value from the **`Popen`** instance.

In [ ]:
from IPython.display import Image
Image(filename='images/signal_parent_group_shell.jpg')